# Grammatical Acceptability

In this notebook, we evaluate the grammaticality of translations using LanguageTool model: https://languagetool.org/fr.
We descibe the procedure in Section 4 of the paper. LanguageTool is open-source grammar tool that is rule-based error corrector, the corpus of rules for French language can be found here: https://community.languagetool.org/rule/list?lang=fr.

Note that below, we install the latest  version of language tool, but installation also requires Java 8.0 or higher. Here you can find detailed instructions for installation: https://github.com/jxmorris12/language_tool_python?tab=readme-ov-file#prerequisites.   
`pip install --upgrade language_tool_python` should take care of downloading LanguageTool.  


Running this notebook takes around 12h and does not require GPU.
This runtime is due to the large number of rules present in the corpus od LanguageTool.

In [ ]:
!pip install --upgrade language_tool_python

In [ ]:
import language_tool_python
tool = language_tool_python.LanguageTool('en-US', config={ 'maxTextLength': 100 })

INFO:language_tool_python.download_lt:Unzipping /tmp/tmp44a6w0m1.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


We do not exclude any rules for French and run checker on all the rules for French.

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
import time
jsonl_file_path = "data_fr/moral_stories_full.jsonl"
json_data = []
with open(jsonl_file_path, 'r') as file:
    for line in file:
        json_data.append(json.loads(line))

In [ ]:
final_df = pd.DataFrame(json_data)
if tool:
    tool.close()

In [ ]:
tool = language_tool_python.LanguageToolPublicAPI('fr-FR')

In [ ]:
cols_to_check = ['norm'] #moral_action,immoral_action,intention,situation,moral_consequence,immoral_consequence
errors=[]
for col in cols_to_check:
    row_n=0
    for value in tqdm(final_df[col]):
        checker_result = tool.check(value)#tool.correct(value)
        time.sleep(0.001)
        if checker_result:
            errors.append({'ID':row_n,"source":value, "check_info":checker_result})
        row_n+=1

 77%|███████▋  | 9299/12000 [2:00:02<31:16,  1.44it/s]

In [ ]:
itemised=[]
for i in errors:
    dict_i={'id': i['ID'],'source':i['source'], 'check_info':{}}
    for j,k in enumerate(i['check_info']):
        if k:
            list_i=[]
            list_i.append(k.category)
            list_i.append(k.context)
            list_i.append(k.errorLength)
            list_i.append(k.matchedText)
            list_i.append(k.message)
            list_i.append(k.offset)
            list_i.append(k.offsetInContext)
            list_i.append(k.replacements)
            list_i.append(k.ruleId)
            list_i.append(k.ruleIssueType)
            dict_i['check_info'][j]=list_i
    itemised.append(dict_i)
jsonl_file_path = "language_tool_norm.jsonl"
with open(jsonl_file_path, 'w', encoding='utf-8') as _file:
    for item in itemised:
        json.dump(item, _file, ensure_ascii=False)
        _file.write('\n')

Examples of suggested corrections

```
{'source': "Kévin a misé sur le Prix de l'Arc de Triomphe et a remporté une grosse somme d'argent, donc maintenant il veut faire des courses hippiques son nouveau passe-temps.",
  'check_info': [Match({'ruleId': 'VIRG_NON_TROUVEE', 'message': 'Une virgule est requise.', 'replacements': [', il'], 'offsetInContext': 43, 'context': "...e grosse somme d'argent, donc maintenant il veut faire des courses hippiques son no...", 'offset': 102, 'errorLength': 3, 'category': 'PONCTUATION_VIRGULE', 'ruleIssueType': 'uncategorized', 'sentence': "Kévin a misé sur le Prix de l'Arc de Triomphe et a remporté une grosse somme d'argent, donc maintenant il veut faire des courses hippiques son nouveau passe-temps."})]},

 {'source': 'Jacques est serveur dans un restaurant et un client nommé Jean vient de commander son steak cher très bien cuit, presque brûlé.',
  'check_info': [Match({'ruleId': 'AI_FR_HYDRA_LEO_MISSING_COMMA', 'message': 'Une virgule semble être requise.', 'replacements': ['cher,'], 'offsetInContext': 43, 'context': '...nommé Jean vient de commander son steak cher très bien cuit, presque brûlé.', 'offset': 92, 'errorLength': 4, 'category': 'MISC', 'ruleIssueType': 'uncategorized', 'sentence': 'Jacques est serveur dans un restaurant et un client nommé Jean vient de commander son steak cher très bien cuit, presque brûlé.'})]},

 {'source': "La petite amie d'Alain souffre de dépression et a laissé leur appartement très désordonné car elle n'a aucune motivation pour le nettoyer.",
  'check_info': [Match({'ruleId': 'PLACE_DE_LA_VIRGULE', 'message': 'Une virgule est conseillée dans ce contexte.', 'replacements': [', car'], 'offsetInContext': 43, 'context': "... laissé leur appartement très désordonné car elle n'a aucune motivation pour le nett...", 'offset': 89, 'errorLength': 4, 'category': 'PONCTUATION_VIRGULE', 'ruleIssueType': 'uncategorized', 'sentence': "La petite amie d'Alain souffre de dépression et a laissé leur appartement très désordonné car elle n'a aucune motivation pour le nettoyer."})]},

 {'source': "Jeanne est une femme âgée qui n'aime pas prendre des douches car c'est difficile pour elle.",
  'check_info': [Match({'ruleId': 'PLACE_DE_LA_VIRGULE', 'message': 'Une virgule est conseillée dans ce contexte.', 'replacements': [', car'], 'offsetInContext': 43, 'context': "... âgée qui n'aime pas prendre des douches car c'est difficile pour elle.", 'offset': 60, 'errorLength': 4, 'category': 'PONCTUATION_VIRGULE', 'ruleIssueType': 'uncategorized', 'sentence': "Jeanne est une femme âgée qui n'aime pas prendre des douches car c'est difficile pour elle."})]},

```

We save the predictions and manually correct the mistakes where needed.